In [ ]:
import data

In [ ]:
sig_data = data.Data("gamutrf_recording_ettus__gain40_1675089435_5735500000Hz_20480000sps.s16.zst")
#sig_data = data.Data("gamutrf_recording_ettus__gain40_1675089435_5735500000Hz_20480000sps.s16.sigmf-meta")

### Export SigMF Data

In [ ]:
sig_data.export_sigmf_data()

### Generate Spectrograms

In [ ]:
n_fft = 1024
time_dim = 512 # equivalent to tune_step_fft parameter in GamutRF
n_samples = n_fft * time_dim
sig_data.generate_spectrograms(n_samples, n_fft)

*Use a 3rd party tool like AnyLabeling to apply labels to the exported spectrograms*

## Import Labelled Spectrograms

In [ ]:
sig_data.import_labelme_spectrograms()

### Export IQ Annotations

In [ ]:
sig_data.export_annotation_iq()

## Import from an S3 bucket

This is a small helper script to download dataset from S3
To use with AWS SSO do: `aws sso login` in the VS Code Terminal

In [ ]:
import boto3
import os

s3_client = boto3.client('s3')

def download_dir(prefix, local, bucket, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

download_dir("01_30_23/", "gamutrf-arl", "gamutrf-arl")